<a href="https://colab.research.google.com/github/DiogoLepri/ASD2_Project/blob/main/ASD_DiagNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ASD2_Project/

/content/drive/MyDrive/ASD2_Project


In [3]:
!pip install pandas numpy matplotlib scikit-learn torch pyprind scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [4]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = True
p_Method = "ASD-DiagNet"

In [5]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)


*****List of patameters****
ROI atlas:  cc200
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  True


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

## Importing the data

In [7]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3])
    else:
        key = '_'.join(f_split[0:2])
    return key

In [8]:
# Define the path to the directory containing time series data.
# The f-string here dynamically inserts the value of p_ROI into the folder name.
data_main_path = f'/content/drive/MyDrive/ASD2_Project/Outputs/cpac/filt_global/rois_{p_ROI}'

# List all files in the directory (each file likely corresponds to a subject's data).
flist = os.listdir(data_main_path)
print(len(flist))  # Print how many files were found.


for f in range(len(flist)):
    flist[f] = get_key(flist[f])


df_labels = pd.read_csv('/content/drive/MyDrive/ASD2_Project/Phenotypic_V1_0b_preprocessed1.csv')#path

# Map the DX_GROUP column so that a value of 1 stays 1 (ASD) and a value of 2 becomes 0 (control)
df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})
print(len(df_labels))

labels = {}  # Initialize an empty dictionary to store labels.

# Iterate over each row in the DataFrame.
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']  # Get the file identifier for the subject.
    y_label = row[1]['DX_GROUP']  # Get the diagnosis label (1 or 0).

    # Skip rows where there is no valid filename.
    if file_id == 'no_filename':
        continue

    # Ensure that each file_id is unique. If a duplicate is found, the program will stop here.
    assert(file_id not in labels)

    # Add the file_id and its corresponding label to the dictionary.
    labels[file_id] = y_label


884
1112


### Helper functions for computing correlations

In [9]:
def get_label(filename):
    # Check that the filename exists in the labels dictionary.
    # If it doesn't, the program will raise an AssertionError(if the expression return false.)
    assert (filename in labels)

    # Return the label associated with this filename from the labels dictionary.
    return labels[filename]



def get_corr_data(filename):
    # Look through all files in the data directory
    for file in os.listdir(data_main_path):
        # If a file's name starts with the given filename string...
        if file.startswith(filename):
            # Read the file into a pandas DataFrame.
            # Files are assumed to be tab-separated values.
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')

    # Temporarily ignore warnings about invalid numerical operations (e.g., division by zero)
    with np.errstate(invalid="ignore"):
        # Compute the correlation matrix for the transposed DataFrame.
        # Transposing (df.T) is done because np.corrcoef expects rows as variables.
        corr = np.nan_to_num(np.corrcoef(df.T))
        # Create a lower-triangular matrix mask.
        # np.tri creates a lower-triangular matrix of ones (True) below the main diagonal,
        # but here k=-1 means the main diagonal is not included.
        # np.invert flips True to False and vice versa, so 'mask' becomes True for
        # the diagonal and upper-triangle, and False for the lower triangle.
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        # Create a masked array 'm' that masks out values where the mask is True.
        m = ma.masked_where(mask == 1, mask)
        # Apply the mask 'm' to the correlation matrix and then compress it.
        # .compressed() returns a 1D array of only the unmasked (i.e., selected) values.
        return ma.masked_where(m, corr).compressed()


def get_corr_matrix(filename):
    # Look through all files in the data directory.
    for file in os.listdir(data_main_path):
        # If a file's name starts with the given filename...
        if file.startswith(filename):
            # Read the file into a DataFrame (assuming tab-separated values).
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    # Again, ignore warnings about invalid numerical operations.
    with np.errstate(invalid="ignore"):
        # Compute the correlation matrix for the transposed DataFrame.
        corr = np.nan_to_num(np.corrcoef(df.T))
        # Return the complete correlation matrix (as a 2D array).
        return corr

def confusion(g_turth, predictions):
    # Compute the confusion matrix from the ground truth and predicted labels.
    # The confusion_matrix function returns a 2x2 matrix for binary classification.
    # We then "ravel" (flatten) it into four numbers:
    # tn: true negatives, fp: false positives, fn: false negatives, tp: true positives.
    tn, fp, fn, tp = confusion_matrix(g_turth, predictions).ravel()

    # Calculate the overall accuracy: (correct predictions) / (total predictions)
    accuracy = (tp + tn) / (tp + fp + tn + fn)

    # Calculate the sensitivity (or recall): the proportion of actual positives correctly identified.
    sensitivity = tp / (tp + fn)

    # Calculate the specificity: the proportion of actual negatives correctly identified.
    specificty = tn / (tn + fp)

    # Return the computed accuracy, sensitivity, and specificity.
    return accuracy, sensitivity, specificty

def get_regs(samplesnames, regnum):
    datas = []
    for sn in samplesnames:
        # For each sample name in samplesnames, extract the correlation data.
        # Here, all_corr is assumed to be a pre-defined dictionary where each key is a sample name.
        # The value all_corr[sn] is expected to be an array or tuple, and we take its first element [0].
        datas.append(all_corr[sn][0])

    # Convert the list of arrays to a NumPy array.
    datas = np.array(datas)

    # Calculate the average correlation value for each region/column across all samples.
    avg = []
    # datas.shape[1] represents the number of regions (or the length of the correlation vector for each sample).
    for ie in range(datas.shape[1]):
        # For each region (column), compute the mean value across all samples.
        avg.append(np.mean(datas[:, ie]))
    avg = np.array(avg)

    # Identify the indices of the 'regnum' regions with the highest average values.
    # argsort returns indices that would sort the array.
    # We take the last 'regnum' indices (which correspond to the highest values) and then reverse their order.
    highs = avg.argsort()[-regnum:][::-1]

    # Similarly, identify the indices of the 'regnum' regions with the lowest average values.
    lows = avg.argsort()[:regnum][::-1]

    # Concatenate the high and low region indices into one array.
    regions = np.concatenate((highs, lows), axis=0)

    return regions

## Helper fnuctions for computing correlations

In [10]:
if not os.path.exists('./correlations_file'+p_ROI+'.pkl'):
    # If the correlations file for this specific p_ROI doesn't exist, we will compute the correlations.

    # Create a progress bar that will show progress for the number of files in flist.
    pbar = pyprind.ProgBar(len(flist))

    # Initialize an empty dictionary to store correlation data and corresponding labels for each file.
    all_corr = {}

    # Loop over each file name in flist.
    for f in flist:
        # Get the label (e.g., diagnosis) for this file.
        lab = get_label(f)
        # Compute the correlation data for this file and store it together with the label in the dictionary.
        all_corr[f] = (get_corr_data(f), lab)
        # Update the progress bar to show that one file has been processed.
        pbar.update()

    print('Corr-computations finished')

    # Save the computed all_corr dictionary to a pickle file.
    # The file name is built dynamically by concatenating './correlations_file', the value of p_ROI, and '.pkl'.
    pickle.dump(all_corr, open('./correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    # If the correlations file already exists, simply load the precomputed data from the file.
    all_corr = pickle.load(open('./correlations_file'+p_ROI+'.pkl', 'rb'))


## Computing eigenvalues and eigenvector

In [11]:
if p_Method == "ASD-DiagNet":
    # Initialize an empty dictionary to hold eigenvalue/eigenvector data for each file.
    eig_data = {}

    # Initialize a progress bar that tracks progress across all files.
    pbar = pyprind.ProgBar(len(flist))

    # Loop through each file in the list.
    for f in flist:
        # Compute the full correlation matrix for the current file.
        d = get_corr_matrix(f)

        # Calculate eigenvalues and eigenvectors of the correlation matrix.
        # np.linalg.eig returns a tuple: (eigenvalues, eigenvectors)
        eig_vals, eig_vecs = np.linalg.eig(d)

        # For each eigenvector, verify that it has unit norm.
        # We use the transpose of eig_vecs (i.e., each column is an eigenvector) for iteration.
        for ev in eig_vecs.T:
            np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

        # Compute the sum of the absolute values of the eigenvalues.
        # This is used to calculate a normalized eigenvalue (a measure of contribution).
        sum_eigvals = np.sum(np.abs(eig_vals))

        # Create a list of tuples, where each tuple contains:
        # (absolute eigenvalue, corresponding eigenvector, normalized eigenvalue)
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i], np.abs(eig_vals[i]) / sum_eigvals)
                     for i in range(len(eig_vals))]

        # Sort these tuples in descending order based on the eigenvalue.
        # This means the eigenpair with the highest eigenvalue comes first.
        eig_pairs.sort(key=lambda x: x[0], reverse=True)

        # Store the sorted eigenpairs in the dictionary using the file name as the key.
        eig_data[f] = {
            'eigvals': np.array([ep[0] for ep in eig_pairs]),       # Sorted eigenvalues
            'norm-eigvals': np.array([ep[2] for ep in eig_pairs]),    # Normalized eigenvalues
            'eigvecs': [ep[1] for ep in eig_pairs]                    # Corresponding eigenvectors
        }

        # Update the progress bar after processing each file.
        pbar.update()


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:32


## Calculating Eros similarity

In [24]:
def norm_weights(sub_flist):
    # Get the number of eigenvalues (dimensions) from one sample.
    # Here, we use the first file in flist (assumed to be defined globally) to get this value.
    num_dim = len(eig_data[flist[0]]['eigvals'])

    # Initialize an array of zeros with a length equal to the number of eigenvalues.
    norm_weights = np.zeros(shape=num_dim)

    # Loop over each file/sample in the provided sublist.
    for f in sub_flist:
        # For each file, add its normalized eigenvalues to our running sum.
        norm_weights += eig_data[f]['norm-eigvals']

    # Return the aggregated normalized eigenvalues.
    return norm_weights


def cal_similarity(d1, d2, weights, lim=None):
    res = 0.0
    if lim is None:
        # If no limit is provided, make a copy of the full weights array.
        weights_arr = weights.copy()
    else:
        # If a limit is provided, take only the first 'lim' elements.
        weights_arr = weights[:lim].copy()
        # Normalize these weights so that they sum to 1.
        weights_arr /= np.sum(weights_arr)

    # Loop through each weight in the (possibly limited) weights array.
    for i, w in enumerate(weights_arr):
        # For each dimension i, compute the inner product of d1[i] and d2[i].
        # Then multiply by the weight 'w' and add it to the result.
        res += w * np.inner(d1[i], d2[i])

    # Return the final weighted similarity.
    return ress

## Defining dataset class

In [13]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None,
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False, regs=None):
        # 'regs' is a parameter that selects specific regions (indices) to be used from the data.
        self.regs = regs

        # Loading the data:
        # If a pickle file is provided, load the precomputed data.
        if pkl_filename is not None:
            if verbose:
                print('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        # Otherwise, if raw data is directly provided, copy it.
        elif data is not None:
            self.data = data.copy()
        else:
            sys.stderr.write('Either PKL file or data is needed!')
            return

        # Determine the list of sample identifiers (filenames or keys).
        # If no specific samples_list is provided, use all keys from self.data.
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]

        # Extract labels for each sample.
        # It is assumed that self.data[f] is a tuple where index 1 holds the label.
        self.labels = np.array([self.data[f][1] for f in self.flist])

        # Create arrays for the full list and for each label separately.
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        # Optionally, you might print out the counts of each label for verification.

        # Data augmentation block:
        # If augmentation is True, we will create additional (augmented) samples.
        if augmentation:
            # Total number of samples will be the original count multiplied by aug_factor.
            self.num_data = aug_factor * len(self.flist)
            self.neighbors = {}  # This dictionary will store "neighbors" for each sample.
            pbar = pyprind.ProgBar(len(self.flist))

            # Compute aggregated normalized weights using the provided samples_list.
            # These weights could be used to measure similarity between eigenvector sets.
            weights = norm_weights(samples_list)  # norm_weights() should return an array of weights.

            # For every sample, find similar neighbors (only within the same class).
            for f in self.flist:
                label = self.data[f][1]
                # Choose candidate neighbors from the same class.
                candidates = set(current_lab0_flist) if label == 0 else set(current_lab1_flist)
                # Remove the current sample from its own candidate list.
                candidates.remove(f)

                # Get the eigenvector data for the current sample.
                eig_f = eig_data[f]['eigvecs']
                sim_list = []  # This will hold tuples of (similarity, candidate)

                # Compute similarity with each candidate using the provided similarity_fn.
                for cand in candidates:
                    eig_cand = eig_data[cand]['eigvecs']
                    sim = similarity_fn(eig_f, eig_cand, weights)
                    sim_list.append((sim, cand))

                # Sort the candidates by similarity score (highest first).
                sim_list.sort(key=lambda x: x[0], reverse=True)
                # Save the top 'num_neighbs' similar candidates as neighbors for sample f.
                self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]
                pbar.update()
        else:
            # If no augmentation is requested, the number of data points equals the number of samples.
            self.num_data = len(self.flist)

    def __getitem__(self, index):
        # The __getitem__ method returns one data sample (and its label).
        # It handles both the original and augmented samples.

        if index < len(self.flist):
            # For indices within the original list, simply retrieve the data.
            fname = self.flist[index]
            # Retrieve the correlation data (stored at index 0 of the tuple).
            data = self.data[fname][0].copy()
            # Select only the regions specified by self.regs.
            data = data[self.regs].copy()
            # Get the label corresponding to this sample.
            label = (self.labels[index],)
            # Convert to torch.FloatTensor and return.
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            # For indices beyond the original list, perform data augmentation.
            # Use modulo to wrap around to an original sample.
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            # Subset the data by the regions of interest.
            d1 = d1[self.regs]
            # Randomly choose one of the precomputed neighbors for f1.
            f2 = np.random.choice(self.neighbors[f1])
            d2, y2 = self.data[f2][0], self.data[f2][1]
            d2 = d2[self.regs]
            # Ensure that both samples have the same label.
            assert y1 == y2
            # Generate a random mixing coefficient.
            r = np.random.uniform(low=0, high=1)
            # The new data is a convex combination of the two neighbors.
            data = r * d1 + (1 - r) * d2
            label = (y1,)
            # Return the augmented sample as torch.FloatTensor.
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        # Return the total number of samples, which might be larger than the original
        # number if augmentation is enabled.
        return self.num_data

## Definig data loader function

In [14]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990,
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        # Store whether the decoder weights are tied to the encoder weights.
        self.tied = tied
        self.num_latent = num_latent

        # Define the encoder layer: transforms input of size 990 to a latent representation of size 200.
        self.fc_encoder = nn.Linear(num_inputs, num_latent)

        # If we are NOT tying weights, we need a separate decoder layer.
        if not tied:
            self.fc_decoder = nn.Linear(num_latent, num_inputs)

        # NOTE: The encoder layer is defined twice in the code.
        # The second definition overwrites the first, so the first one could be removed.
        self.fc_encoder = nn.Linear(num_inputs, num_latent)

        # Define the classifier head.
        # If use_dropout is True, add a dropout layer before the classification.
        if use_dropout:
            self.classifier = nn.Sequential(
                nn.Dropout(p=0.5),  # Drop 50% of the neurons randomly during training for regularization.
                nn.Linear(self.num_latent, 1),  # Linear layer mapping latent features to 1 output (e.g., for binary classification).
            )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(self.num_latent, 1),
            )

    def forward(self, x, eval_classifier=False):
        # Pass input through the encoder to get the latent representation.
        x = self.fc_encoder(x)
        # Apply the tanh activation function to introduce non-linearity.
        x = torch.tanh(x)

        # If we are evaluating the classifier (e.g., during a classification task),
        # pass the latent features through the classifier head.
        if eval_classifier:
            x_logit = self.classifier(x)
        else:
            x_logit = None

        # For the reconstruction part:
        if self.tied:
            # Tied weights: Use the transposed weights of the encoder as the decoder.
            # This is equivalent to doing: x = x @ (encoder_weights)^T
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            # Use a separate decoder layer.
            x = self.fc_decoder(x)

        # Return a tuple:
        # - x: the reconstructed input.
        # - x_logit: the output of the classifier (if eval_classifier is True, else None).
        return x, x_logit

# Instantiate the model
mtae = MTAutoEncoder()

# Print the model architecture (if you run this cell, you'll see the details of the layers)
mtae

## Defining Autoencoder class

In [15]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990,
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        # Save key parameters for later use in the model.
        self.tied = tied               # Controls whether the decoder's weights are "tied" to the encoder's.
        self.num_latent = num_latent   # Dimension of the latent (compressed) representation.

        # --- ENCODER DEFINITION ---
        # Define a linear layer that maps the input (of size 990) to the latent space (of size 200).
        self.fc_encoder = nn.Linear(num_inputs, num_latent)

        # --- DECODER DEFINITION ---
        # If the weights are not tied, then define a separate linear layer for the decoder.
        if not tied:
            self.fc_decoder = nn.Linear(num_latent, num_inputs)

        # Note: The encoder layer is redefined immediately here, which overwrites the previous definition.
        # This redundancy may be unintentional, but the result is that the model uses this new definition.
        self.fc_encoder = nn.Linear(num_inputs, num_latent)

        # --- CLASSIFIER DEFINITION ---
        # The classifier takes the latent representation and maps it to a single output.
        # It can optionally use dropout to help prevent overfitting.
        if use_dropout:
            self.classifier = nn.Sequential(
                nn.Dropout(p=0.5),            # Randomly drops 50% of units during training.
                nn.Linear(self.num_latent, 1)  # Linear layer that maps latent space to one output.
            )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(self.num_latent, 1)
            )

    def forward(self, x, eval_classifier=False):
        # --- ENCODING ---
        # Pass the input 'x' through the encoder layer.
        x = self.fc_encoder(x)
        # Apply the tanh activation function to introduce non-linearity.
        x = torch.tanh(x)

        # --- CLASSIFICATION (OPTIONAL) ---
        # If we are in a mode to evaluate classification, pass the latent features through the classifier.
        if eval_classifier:
            x_logit = self.classifier(x)
        else:
            x_logit = None

        # --- DECODING / RECONSTRUCTION ---
        if self.tied:
            # When using tied weights, the decoder uses the transpose of the encoder's weight matrix.
            # This essentially reverses the encoding operation.
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            # Use the separate decoder layer if weights are not tied.
            x = self.fc_decoder(x)

        # The function returns a tuple:
        # - x: the reconstructed input.
        # - x_logit: the classification output (if computed) or None.
        return x, x_logit

# Create an instance of the autoencoder.
mtae = MTAutoEncoder()

# Display the model's architecture.
mtae

MTAutoEncoder(
  (fc_encoder): Linear(in_features=990, out_features=200, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=200, out_features=1, bias=True)
  )
)

## Defining training and testing functions

In [16]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()  # Set the model to training mode.
    train_losses = []  # To store losses for each batch.

    # Loop over each batch from the train_loader.
    for i, (batch_x, batch_y) in enumerate(train_loader):
        # Skip batches that are not full-sized.
        if len(batch_x) != batch_size:
            continue

        # If p_bernoulli is provided, create a random binary mask (noise) for data corruption.
        if p_bernoulli is not None:
            # Only need to create this mask once, when i == 0.
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device) * p_bernoulli
            # Generate random 0/1 values based on the probability p_bernoulli.
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        # Move the input data and labels to the device (CPU or GPU).
        data, target = batch_x.to(device), batch_y.to(device)

        # Reset gradients before the backward pass.
        optimizer.zero_grad()

        # *** Autoencoder (AE) Reconstruction Loss ***
        # This branch handles the autoencoder part (reconstructing the input data).
        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                # Multiply data by the random binary mask to add noise.
                rec_noisy, _ = model(data * rand_bernoulli, False)
                # Compute the reconstruction loss comparing the noisy reconstruction with the clean data.
                loss_ae = criterion_ae(rec_noisy, data) / len(batch_x)
            else:
                # If no noise is added, just reconstruct the original data.
                rec, _ = model(data, False)
                loss_ae = criterion_ae(rec, data) / len(batch_x)

        # *** Classifier Loss ***
        # This branch handles the classifier part (predicting the labels).
        if mode in ['both', 'clf']:
            # Get the reconstruction and the classifier logits from the model.
            rec_clean, logits = model(data, True)
            # Compute the classification loss using the predicted logits and the true labels.
            loss_clf = criterion_clf(logits, target)

        # Combine the losses based on the chosen mode.
        if mode == 'both':
            # When both tasks are being trained, add the AE loss and the classifier loss,
            # with the classifier loss scaled by lam_factor.
            loss_total = loss_ae + lam_factor * loss_clf
            # Save the losses for later analysis.
            train_losses.append([loss_ae.detach().cpu().numpy(),
                                 loss_clf.detach().cpu().numpy()])
        elif mode == 'ae':
            loss_total = loss_ae
            train_losses.append([loss_ae.detach().cpu().numpy(), 0.0])
        elif mode == 'clf':
            loss_total = loss_clf
            train_losses.append([0.0, loss_clf.detach().cpu().numpy()])

        # *** Backward Pass and Optimization ***
        loss_total.backward()  # Compute gradients.
        optimizer.step()       # Update the model's parameters.

    # Return a list of the losses for each batch in this epoch.
    return train_losses

def test(model, criterion, test_loader,
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    all_predss = []  # To store all predictions for classification.

    # If evaluating the classifier, we'll keep track of true and predicted values.
    if eval_classifier:
        y_true, y_pred = [], []

    # No gradient computation during testing.
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode.

        # Loop over the test data batches.
        for i, (batch_x, batch_y) in enumerate(test_loader, 1):
            # If a specific number of batches is desired, limit the loop.
            if num_batch is not None:
                if i >= num_batch:
                    continue

            # Move the data to the appropriate device.
            data = batch_x.to(device)
            # Get the reconstructed data and classifier logits from the model.
            rec, logits = model(data, eval_classifier)

            # Compute the reconstruction loss and add it to the total loss.
            test_loss += criterion(rec, data).detach().cpu().numpy()
            n_test += len(batch_x)

            # If classifier evaluation is turned on, process classification outputs.
            if eval_classifier:
                # Apply a sigmoid to convert logits to probabilities.
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                # Initialize predictions as ones (class 1).
                preds = np.ones_like(proba, dtype=np.int32)
                # Assign class 0 for probabilities below 0.5.
                preds[proba < 0.5] = 0
                all_predss.extend(preds)  # Collect predictions.

                # Convert true labels to a NumPy array.
                y_arr = np.array(batch_y, dtype=np.int32)

                # Count correct predictions.
                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())

        # Compute additional metrics using a custom confusion function.
        mlp_acc, mlp_sens, mlp_spef = confusion(y_true, all_predss)

    # Return the computed accuracy, sensitivity, and specificity.
    return mlp_acc, mlp_sens, mlp_spef  # ,correct/n_test (this part is commented out)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [54]:
if p_Method == "ASD-DiagNet" and p_mode == "whole":

    # Get the number of correlations (features) from the first sample.
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ", num_corr)

    start = time.time()  # Record the start time for overall runtime.

    # Set training hyperparameters.
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001  # Learning rates for the autoencoder and classifier.
    num_epochs = 25  # Total number of training epochs.

    p_bernoulli = None  # No noise injection in this configuration.
    augmentation = p_augmentation  # Flag for data augmentation.
    use_dropout = False  # Whether to use dropout in the classifier.

    # Augmentation and similarity parameters.
    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr / 4)  # Initial latent space size based on a fraction of the total correlations.
    print(n_lat)
    start = time.time()  # (Re)start the timer for training.

    # Print the current configuration.
    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor,
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')

    # Create a similarity function with a fixed limit using functools.partial.
    sim_function = functools.partial(cal_similarity, lim=lim4sim)

    crossval_res_kol = []  # List to store cross-validation results.

    # Get labels for all files using a helper function.
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)  # Convert the file list to a NumPy array.
    kk = 0  # Initialize a counter (though it will be redefined in the loop below).

    # Repeat the entire cross-validation process 10 times.
    for rp in range(10):
        # Set up stratified k-fold cross-validation.
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)

        # Shuffle the list of files to randomize the splits.
        np.random.shuffle(flist)

        # Update labels after shuffling.
        y_arr = np.array([get_label(f) for f in flist])

        # Loop through each fold.
        for kk, (train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            # Select training and testing samples based on indices.
            train_samples, test_samples = flist[train_index], flist[test_index]

            # Set verbose to True for the first fold to see detailed output.
            verbose = (True if (kk == 0) else False)

            # Select regions (features) from the training samples.
            regions_inds = get_regs(train_samples, int(num_corr / 4))

            # Determine the number of inputs for the model based on the selected regions.
            num_inpp = len(regions_inds)
            # Set the latent dimension to half of the number of inputs.
            n_lat = int(num_inpp / 2)

            # Create a DataLoader for training data.
            train_loader = get_loader(
                data=all_corr,
                samples_list=train_samples,
                batch_size=batch_size,
                mode='train',
                augmentation=augmentation,
                aug_factor=aug_factor,
                num_neighbs=num_neighbs,
                eig_data=eig_data,
                similarity_fn=sim_function,
                verbose=verbose,
                regions=regions_inds
            )

            # Create a DataLoader for testing data.
            test_loader = get_loader(
                data=all_corr,
                samples_list=test_samples,
                batch_size=batch_size,
                mode='test',
                augmentation=False,
                verbose=verbose,
                regions=regions_inds
            )

            # Initialize the model with tied weights.
            model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)  # Move the model to GPU/CPU.

            # Define the loss functions:
            # Autoencoder reconstruction loss.
            criterion_ae = nn.MSELoss(reduction='sum')
            # Classifier loss (binary cross-entropy with logits).
            criterion_clf = nn.BCEWithLogitsLoss()

            # Set up the optimizer to update the encoder and classifier parameters.
            optimizer = optim.SGD(
                [{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                 {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                momentum=0.9
            )

            # Train the model over multiple epochs.
            for epoch in range(1, num_epochs + 1):
                # For the first 20 epochs, train both autoencoder and classifier.
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    # After 20 epochs, train only the classifier part.
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')

            # Evaluate the model on the test set.
            res_mlp = test(model, criterion_ae, test_loader, eval_classifier=True)
            # Print test results for this fold.
            print(test(model, criterion_ae, test_loader, eval_classifier=True))

            # Save the result for this fold.
            crossval_res_kol.append(res_mlp)

        # After finishing all folds, print the average results for this repeat.
        print("averages:")
        print(np.mean(np.array(crossval_res_kol), axis=0))
        finish = time.time()
        # Print the running time for this repeat.
        print(finish - start)

Per Site p_mode = "percenter" and p_center = " "

In [ ]:
p_ROI = "cc200"
p_center = "CMU"  # Change to the desired center
p_mode = "percenter" #Change to percenter
p_Method = "ASD-DiagNet"
p_fold = 5  # Desired number of folds for larger centers
p_augmentation = True

In [37]:
if p_Method == "ASD-DiagNet" and p_mode == "percenter":
    # Determine the number of correlations (features) available from the first file.
    num_corr = len(all_corr[flist[0]][0])

    # List all files in the main data path and process them to extract a key.
    flist = os.listdir(data_main_path)
    flist = [get_key(f) for f in flist]

    # Group files by centers (e.g., different imaging centers)
    centers_dict = {}
    for f in flist:
        key = f.split('_')[0]  # Assume the center ID is the first part of the filename.
        centers_dict.setdefault(key, []).append(f)

    # Select files that belong to the current center (p_center).
    flist = np.array(centers_dict[p_center])

    # Get labels for each file (e.g., indicating ASD or not) using a helper function.
    y_arr = np.array([get_label(f) for f in flist])

    # Determine the number of splits for cross-validation based on class balance.
    unique_labels, counts = np.unique(y_arr, return_counts=True)
    new_n_splits = min(p_fold, counts.min())  # Ensure every fold has at least one sample of each class.

    if new_n_splits < 2:
        print(f"Skipping center {p_center} due to insufficient samples in one class.")
    else:
        print(f"Using {new_n_splits}-fold cross-validation for center {p_center}.")

        start = time.time()  # Start timing the process.

        # Set training parameters.
        batch_size = 8
        learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
        num_epochs = 25
        p_bernoulli = None  # This can be used for noise injection.
        augmentation = p_augmentation  # Whether to perform data augmentation.
        use_dropout = False
        aug_factor = 2
        num_neighbs = 5
        lim4sim = 2
        n_lat = int(num_corr / 4)  # Initial latent dimension (will be updated later).

        # Create a partial function for calculating similarity with a fixed limit.
        sim_function = functools.partial(cal_similarity, lim=lim4sim)
        all_rp_res = []  # To store results for each repeat.

        # Repeat the entire cross-validation process 10 times for stability.
        for rp in range(10):
            print(f"Running repeat {rp + 1} for center {p_center}...")
            crossval_res_kol = []  # Store results for each fold in this repeat.

            # Set up stratified k-fold cross-validation.
            kf = StratifiedKFold(n_splits=new_n_splits)
            for kk, (train_index, test_index) in enumerate(kf.split(flist, y_arr)):
                # Get training and testing samples based on indices.
                train_samples, test_samples = flist[train_index], flist[test_index]

                # For the first fold, set verbose to True to get extra output.
                verbose = (kk == 0)

                # Determine which regions (features) to use; here, it's a subset of the correlations.
                regions_inds = get_regs(train_samples, int(num_corr / 4))
                num_inpp = len(regions_inds)  # Number of input features for the model.
                n_lat = int(num_inpp / 2)      # Set the latent space size to half the number of inputs.

                # Create DataLoaders for training and testing using a helper function.
                train_loader = get_loader(
                    data=all_corr,
                    samples_list=train_samples,
                    batch_size=batch_size,
                    mode='train',
                    augmentation=augmentation,
                    aug_factor=aug_factor,
                    num_neighbs=num_neighbs,
                    eig_data=eig_data,
                    similarity_fn=sim_function,
                    verbose=verbose,
                    regions=regions_inds
                )

                test_loader = get_loader(
                    data=all_corr,
                    samples_list=test_samples,
                    batch_size=batch_size,
                    mode='test',
                    augmentation=False,
                    verbose=verbose,
                    regions=regions_inds
                )

                # Initialize the model with tied weights, using the determined number of input features and latent dimension.
                model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
                model.to(device)  # Move the model to GPU/CPU.

                # Define loss functions for autoencoder (reconstruction) and classifier.
                criterion_ae = nn.MSELoss(reduction='sum')
                criterion_clf = nn.BCEWithLogitsLoss()

                # Set up the optimizer to update encoder and classifier parameters with separate learning rates.
                optimizer = optim.SGD(
                    [{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                     {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                    momentum=0.9
                )

                # Training loop for the specified number of epochs.
                for epoch in range(1, num_epochs + 1):
                    if epoch <= 20:
                        # In early epochs, train both the autoencoder (AE) and classifier.
                        train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                    else:
                        # In later epochs, focus only on training the classifier.
                        train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')

                # Evaluate the model on the test set.
                res_mlp = test(model, criterion_ae, test_loader, eval_classifier=True)
                crossval_res_kol.append(res_mlp)

            # Print the average result for this repeat (across all folds).
            print(f"Result of repeat {rp + 1} for center {p_center}: {np.mean(np.array(crossval_res_kol), axis=0)}")
            all_rp_res.append(np.mean(np.array(crossval_res_kol), axis=0))

        # Print the overall average result across all repeats.
        print(f"Average result for 10 repeats for center {p_center}: {np.mean(np.array(all_rp_res), axis=0)}")
        finish = time.time()
        print(f"Total running time for center {p_center}: {finish - start:.2f} seconds")

Using 2-fold cross-validation for center CMU.
Running repeat 1 for center CMU...
Computing neighbors for augmentation...


TypeError: cal_similarity() missing 1 required positional argument: 'weights'

Result Paper - Result Mine:

Accuracy%


---



Caltech: 52.8% - 62.75%

CMU: 68.5% - ERROR

KKI: 69.5% - 69.3%

Leuven: 61.3% - 54.1%

Maxmun: 48.6% - 63.9%

NYU: 68.0% - 67.9%

OHSU: 82% - 73.3%

Olin: 65.1% - 68.8%

Pitt: 67.8% - 63.3%

SBL: 51.6% - 43.6%

SDSU: 63.0% - 64.0%

Stanford: 64.2% - 64.9%

Trinity: 54.1% - 53.1%

UCLA: 73.2% - 67.1%

USM: 68.2% - 68.2%

UM: 63.8% - 65.7%

Yale: 63.6% - 64.1%
